In [20]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '..')

from pairing import Reader, Extractor, Classifier
import definition
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score, confusion_matrix

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Loading and Preprocessing

In [21]:
train_ratio = 0.5

raw_data = Reader.read_file(definition.DATA_PAIRED_SAMPLE)
index = np.arange(len(raw_data))
np.random.shuffle(index)
index_train = index[:int(train_ratio * len(raw_data))]
index_test = index[int(train_ratio * len(raw_data)):]
raw_data_train = np.array(raw_data)[index_train].tolist()
raw_data_test = np.array(raw_data)[index_test].tolist()

data_train = Extractor.extract_data(raw_data_train)
data_test = Extractor.extract_data(raw_data_test)

100%|███████████████████████████████████████████████████████████████████████████| 1349/1349 [00:00<00:00, 34608.97it/s]


In [22]:
data_train.head()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
0,8,9,9,1,15,2,9,0,0
1,6,6,9,1,5,1,9,3,0
2,3,3,9,1,3,1,9,6,0
3,1,1,9,1,5,1,9,8,1
4,1,1,9,1,4,1,0,1,1


In [23]:
data_train.describe()

,dist_endpoint,dist_start,len_aspect_char,len_aspect_word,len_sentiment_char,len_sentiment_word,position_aspect,position_sentiment,target
count,9339.000000,9339.000000,9339.000000,9339.000000,9339.000000,9339.000000,9339.000000,9339.000000,9339.000000
mean,7.913374,8.280758,7.946568,1.338687,8.296606,1.516758,10.820323,12.012100,0.318128
std,8.798017,8.827648,3.560575,0.591376,4.367886,0.708856,12.046638,12.283724,0.465775
min,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,2.000000,5.000000,1.000000,5.000000,1.000000,2.000000,3.000000,0.000000
50%,5.000000,5.000000,8.000000,1.000000,7.000000,1.000000,7.000000,8.000000,0.000000
75%,11.000000,11.000000,10.000000,2.000000,11.000000,2.000000,16.000000,17.000000,1.000000
max,67.000000,67.000000,34.000000,6.000000,33.000000,6.000000,64.000000,69.000000,1.000000


In [24]:
X_train = data_train.drop('target', axis=1)
X_test = data_test.drop('target', axis=1)
y_train = data_train['target']
y_test = data_test['target']

## Training

In [25]:
model = Classifier()

In [26]:
model.fit(X_train, y_train)
model.model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [27]:
pred = model.predict(X_test)

In [32]:
print("f1_0 : {}".format(f1_score(y_test, pred, pos_label=0)))
print("f1_1 : {}".format(f1_score(y_test, pred, pos_label=1)))
print("f1_a : {}".format(f1_score(y_test, pred, average='macro')))

f1_0 : 0.943071161048689
f1_1 : 0.8654390934844193
f1_a : 0.9042551272665542


In [29]:
Classifier.generate_confusion_matrix_table(y_test, pred)

,predicted_0,predicted_1
true_0,6295,256
true_1,504,2444
